# Day 11 Lab 2 | Troubleshooting Static Routes

![inicio](photos/day11pt2/inicio.jpg)

Before we get started troubleshooting, we should confirm the problem. PC1 and PC2 should not be able to ping each other.

Let's send the ping:

![ping failed](photos/day11pt2/pingfailed.jpg)

We can see that the ping does not work.  
Although, the missconfiguration is on routers, let's check PCs configurations:

![ip config all](photos/day11pt2/ipconfigall.jpg)

Let's see if we can reach the Default Gateways, since to reach other networks we need to be able to reach Default Gateway.  
As I can see, for PC1 the default gateway is 192.168.1.254  
Let's ping it:

![ping gateway done](photos/day11pt2/pinggateway.jpg)

No problems here.

Let's go on R1 now and see if we can find the issue.

In R1 CLI enter privileged exect mode and check if the interface configuration is okay.

![ip interface](photos/day11pt2/ipinterface.jpg)

The addresses are correct and both interfaces are UP/UP, so the interface configuration is fine. 

Let's check the routing table:

![static](photos/day11pt2/static.jpg)

Here is the Static route for the 192.168.3.0/24 network. It says via 192.168.12.3, but actually the next hop address should be 192.168.12.2, which is the IP address of R2's g0/0 interface.

Show running-config and then include pipe simbol to filter the output, include and then ip route:

![ip route](photos/day11pt2/iproute.jpg)

Here you can see the configuration statement.  
I'm going to copy it and then go to global config mode and paste in the configuration statement that I just copied and use the keyboard shortcut CTRL+A to go to the beginning of the line, type NO followed by a space and then hit Enter.  
And that deletes that configuration statement.

![cancel configuration](photos/day11pt2/noconfig.jpg)

Let's check

Type in the command in blue and hit Enter. Nothing is shown, meaning that it's gone:

![gone](photos/day11pt2/gone.jpg)

Now I will paste in the configuration statement once more and change the 3 to a 2, hit Enter and use up arrow to go back to the show-running config command and now you can see we have the correct configuration statement:

![correct](photos/day11pt2/correct.jpg)

Let's check the routing table one last time:

![fixed](photos/day11pt2/fixed.jpg)

Ok, there is the correct route, so we have fixed the problem on R1.

Now let's move on to R2

The IP addresses are both correct and the interfaces are both UP/UP, so no problems here

![r2](photos/day11pt2/r2.jpg)

Let's check the routing table:

![routing table](photos/day11pt2/routingtable.jpg)

We have two static routes, as expected. 192.168.1.0/24 via 192.168.12.1 is correct  
and 192.168.3.0/24, with an exit interface GigabitEthernet0/0 <- there's our problem

To reach the 192.168.3.0/24 network, R2 should use the GigabitEthernet0/1 interface not 0/0

Let's see what happens if I enter the correct route without removing the incorrect route:

![routes](photos/day11pt2/routes.jpg)

```commandline
configure terminal  
ip route 192.168.3.0 255.255.255.0 g0/1  
do show ip route

We can see that both routes are kept in the routing table, the second does not override the first one.  
Because both are in the routing table, the router will 'load-balance' over the two routes, sending some packets out of g0/0, and some packets out of g0/1. Sometimes load-balancing is a good thing, but not in this case, because the g0/0 interface is totally wrong, so we should remove that configuration statement.

```commandline
do show run | include ip route

Copy the IP route that we need to delete, paste it in the new line, press CTRL+A to move to the beginning of the line and type NO, press Enter.  
Now let's check are the changes that we needed done correctly by typing do show ip route:

![r2 done](photos/day11pt2/r2done.jpg)

Ok, now we only have one route to 192.168.3.0/24, using the correct interface 0/1

Finally, let's look for an issue on R3.

![r3 wrong](photos/day11pt2/r3wrong.jpg)

We can see that the IP address is incorrect. It should be 192.168.13.3

Let's fix that

```commandline
int g0/0  


Now, unlike when configuring routes, we don't have to remove the current configuration first. IF we enter a new IP address, it will overwrite the current IP address:

![new ip](photos/day11pt2/newip.jpg)

Let's check the configuration again, to see if it really changed the IP address:

```commandline
do show-running config

![correct ip](photos/day11pt2/goodip.jpg)

As you cna see the previous configuration was automatically overwritten, we did not need to delete it first.

Now, that we have finished the misconfigurations, let's get back to PC1 and try out that ping. The first ping, or two, might fail, but after that the ping should be sent. 

![lab done](photos/day11pt2/labdone.jpg)

Alright, after finding and fixing misconfigurations on R1, R2 and R3, PC1 is able to ping PC2.

Lab finished!